> ## A limpeza dos dados foi devidamente realizada no notebook referente à [Parte 1](https://colab.research.google.com/drive/1HU_OmJrgzpN7gZN_yOK8psxw7D_ufZNq?usp=sharing). Confira antes de prosseguir neste artefato.

## EDA Parte 2: Exploração inicial dos dados

O intuíto desse notebook é fazer uma análise exploratória inicial, observado características dos dados que serão melhor apontadas no resultado final da análise e identificando comportamentos inesperados, não usuais ou interessantes nos mesmos.

In [46]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1Eq4AMr9VKFt1H8p4qfYkTMagBecDoBX0
!gdown https://drive.google.com/uc?id=17B7Zy7yPLRJqThgK5OY9Z_ztMNKnzSmx

Downloading...
From: https://drive.google.com/uc?id=1Eq4AMr9VKFt1H8p4qfYkTMagBecDoBX0
To: /content/PRIMEIRO_TURNO_CG_dados_tratados.csv
100% 22.4M/22.4M [00:00<00:00, 167MB/s]
Downloading...
From: https://drive.google.com/uc?id=17B7Zy7yPLRJqThgK5OY9Z_ztMNKnzSmx
To: /content/SEGUNDO_TURNO_CG_dados_tratados.csv
100% 1.14M/1.14M [00:00<00:00, 156MB/s]


In [47]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [48]:
color_pallete = ['#F05B13', '#1319F0', '#4EF013']

# Observando dados referentes ao primeiro turno

In [49]:
df_primeiro_turno = pd.read_csv("PRIMEIRO_TURNO_CG_dados_tratados.csv").drop(columns=["Unnamed: 0", "NR_ZONA.1", "NR_SECAO.1"])
df_primeiro_turno.head()

,NM_MUNICIPIO,NR_ZONA,NR_SECAO,NR_TURNO,NR_LOCAL_VOTACAO,NR_PARTIDO,SG_PARTIDO,DS_CARGO_PERGUNTA,DS_TIPO_VOTAVEL,NM_VOTAVEL,NR_LATITUDE,NR_LONGITUDE,NM_BAIRRO,NM_LOCAL_VOTACAO
0,CAMPINA GRANDE,72,66,1.0,1198.0,30.0,NOVO,Presidente,Nominal,FELIPE D'AVILA,-7.226579,-35.918852,CONJUNTO SEVERINO CABRAL,ESCOLA ESTADUAL DE 1. E 2. GRAUS SEVERINO CABRAL
1,CAMPINA GRANDE,72,66,1.0,1198.0,-1.0,#NULO#,Presidente,Branco,Branco,-7.226579,-35.918852,CONJUNTO SEVERINO CABRAL,ESCOLA ESTADUAL DE 1. E 2. GRAUS SEVERINO CABRAL
2,CAMPINA GRANDE,72,66,1.0,1198.0,-1.0,#NULO#,Presidente,Nulo,Nulo,-7.226579,-35.918852,CONJUNTO SEVERINO CABRAL,ESCOLA ESTADUAL DE 1. E 2. GRAUS SEVERINO CABRAL
3,CAMPINA GRANDE,72,66,1.0,1198.0,15.0,MDB,Presidente,Nominal,SIMONE TEBET,-7.226579,-35.918852,CONJUNTO SEVERINO CABRAL,ESCOLA ESTADUAL DE 1. E 2. GRAUS SEVERINO CABRAL
4,CAMPINA GRANDE,72,66,1.0,1198.0,12.0,PDT,Presidente,Nominal,CIRO GOMES,-7.226579,-35.918852,CONJUNTO SEVERINO CABRAL,ESCOLA ESTADUAL DE 1. E 2. GRAUS SEVERINO CABRAL


## Densidade de votos por local de votação

O código abaixo gera um mapa de densidade que mostra a distribuição geográfica da densidade de votos por local de votação em Campina Grande, usando as coordenadas de latitude e longitude. Cada ponto no mapa representa um local de votação e a densidade de votos é indicada pela intensidade da cor.

In [50]:
coord_counts = df_primeiro_turno.groupby(['NR_LATITUDE', 'NR_LONGITUDE']).size().reset_index(name='Contagem')

fig = px.density_mapbox(coord_counts, lat='NR_LATITUDE', lon='NR_LONGITUDE', z='Contagem', radius=30,
                        center=dict(lat=-7.2307, lon=-35.8811), zoom=12,
                        hover_name='Contagem', hover_data=['NR_LATITUDE', 'NR_LONGITUDE'],
                        title='Distribuição Geográfica da densidade de votos por local de votação em Campina Grande',
                        mapbox_style='open-street-map')

fig.show()

A questão mais perceptiva na visualização acima é que a maior densidade de votos se encontra nas regiões vinculadas ao centro da cidade, enquanto nas periferias essa densidade é esmaecida.

> O bairro Santa Terezinha apresenta os menores indicadores segundo a [ANÁLISE RETROSPECTIVA E
SITUAÇÃO ATUAL](https://campinagrande.pb.gov.br/wp-content/uploads/2020/07/1-An-lise-Retrospectiva-e-Situa%E2%80%A1%C3%86o-Atual.pdf) sobre dados socioeconômicos da cidade de Campina Grande.

## Distribuição das Zonas Eleitorais

A visualização abaixo tem o objetivo de apresentar de forma gráfica e interativa a distribuição geográfica das zonas eleitorais na cidade de Campina Grande. Cada ponto no mapa representa um local de votação, e a cor e tamanho dos pontos indicam a zona eleitoral à qual o local pertence e a quantidade de coordenadas associadas, respectivamente. Isso pode ser útil para visualizar a dispersão das zonas eleitorais na cidade e pode fornecer informações importantes para análises e planejamentos eleitorais.

In [51]:
agrupamento_zona = df_primeiro_turno.groupby(['NR_ZONA', 'NR_LATITUDE', 'NR_LONGITUDE']).size().reset_index(name='Contagem')
agrupamento_zona['Zona'] = agrupamento_zona['NR_ZONA'].astype(str)

fig = px.scatter_mapbox(agrupamento_zona, lat='NR_LATITUDE', lon='NR_LONGITUDE', color='Zona', size='Contagem',
                        center=dict(lat=-7.2307, lon=-35.8811), zoom=12,
                        hover_name='NR_ZONA', hover_data=['Contagem'],
                        title='Distribuição das Zonas Eleitorais na cidade de Campina Grande',
                        mapbox_style='open-street-map',
                        color_discrete_sequence=color_pallete,
                        color_discrete_map={i: f'Zona {i}' for i in agrupamento_zona['NR_ZONA'].unique()},
                        size_max=30)

fig.update_layout(
    legend=dict(
        title='Zonas Eleitorais',
        orientation='h',
        itemsizing='constant'
    )
)

fig.show()

## Quantidade de locais de votação por bairros da cidade

Esta visualização tem o objetivo de apresentar de forma clara e informativa a contagem de locais de votação em cada bairro, divididos por zona eleitoral. É útil para entender a distribuição dos locais de votação na cidade e pode fornecer insights importantes para planejamento e logística nas eleições. A legenda de cores indica a qual zona eleitoral cada barra está associada.

In [52]:
agrupamento_zona_bairro = df_primeiro_turno[['NR_ZONA', 'NM_BAIRRO', 'NM_LOCAL_VOTACAO']].drop_duplicates().groupby(['NR_ZONA', 'NM_BAIRRO']).size().reset_index(name='Contagem')

agrupamento_zona_bairro['Zona'] = agrupamento_zona_bairro['NR_ZONA'].astype(str)

fig = px.bar(agrupamento_zona_bairro, x='NM_BAIRRO', y='Contagem', color='Zona',
             title='Contagem de Locais de Votação por Bairro e Zona Eleitoral',
             labels={'Contagem': 'Número de Locais de Votação', "NM_BAIRRO": 'Bairro', "Zona": "Zona Eleitoral"},
             color_discrete_sequence=color_pallete)

fig.show()

## Contagem de votos nos Top 10 partidos por bairro

In [53]:
cores_partidos = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2',
                  '#7f7f7f', '#bcbd22', '#17becf', '#b5bd61', '#c49c94', '#f7b6d2', '#c7c7c7',
                  '#dbdb8d', '#9edae5', '#98df8a', '#ff9896', '#aec7e8', '#ffbb78', '#c5b0d5',
                  '#ff9896', '#ff7f0e', '#aec7e8', '#ffbb78', '#98df8a', '#c49c94', '#dbdb8d',
                  '#ffbb78', '#ff9896', '#aec7e8', '#ff7f0e', '#98df8a']

In [54]:
agrupamento_partido_bairro = df_primeiro_turno.groupby(['SG_PARTIDO', 'NM_BAIRRO']).size().reset_index(name='Contagem')

agrupamento_partido_bairro = agrupamento_partido_bairro.sort_values(by='Contagem', ascending=False)

top_10_partidos_bairro = agrupamento_partido_bairro.groupby('NM_BAIRRO').head(10)

fig = px.bar(top_10_partidos_bairro, x='NM_BAIRRO', y='Contagem', color='SG_PARTIDO',
             title='Contagem de Votos para os Top 10 Partidos por Bairro',
             labels={'Contagem': 'Número de Votos', "NM_BAIRRO": 'Bairro', "SG_PARTIDO": "Partido"},
             color_discrete_sequence=px.colors.qualitative.Plotly)

fig.update_layout(legend_title='Partido', barmode='stack')
fig.show()


Essa visualização é útil para entender a distribuição de votos para os top 10 partidos em cada bairro, proporcionando insights sobre as preferências políticas em diferentes regiões da cidade. A empilhamento das barras permite comparar a contagem total de votos por bairro e observar como cada partido contribui para esse total.

## Contagem de votos brancos e nulos por bairro

Essa visualização é útil para entender a distribuição de votos brancos e nulos por zona eleitoral em cada bairro. A empilhamento das barras permite comparar a contagem total de votos brancos e nulos por bairro e observar como cada tipo de voto contribui para esse total. A legenda de cores indica se a barra representa um voto Branco ou Nulo.

In [55]:
df_brancos_nulos_zona = df_primeiro_turno[df_primeiro_turno['DS_TIPO_VOTAVEL'].isin(['Branco', 'Nulo'])]

agrupamento_zona_tipo_voto = df_brancos_nulos_zona.groupby(['NM_BAIRRO', 'DS_TIPO_VOTAVEL']).size().reset_index(name='Contagem')

fig = px.bar(agrupamento_zona_tipo_voto, x='NM_BAIRRO', y='Contagem', color='DS_TIPO_VOTAVEL',
             title='Contagem de Votos Brancos e Nulos por Zona Eleitoral',
             labels={'Contagem': 'Número de Votos', "NM_BAIRRO": 'Zona Eleitoral', "DS_TIPO_VOTAVEL": 'Tipo de Voto'},
             color_discrete_sequence=px.colors.qualitative.Plotly)

fig.update_layout(legend_title='Tipo de Voto')

fig.show()

## Calculando as correlações entre as colunas

In [56]:
import plotly.figure_factory as ff

correlation_matrix = df_primeiro_turno.drop(columns=["NR_TURNO"]).corr()

fig = ff.create_annotated_heatmap(z=correlation_matrix.values,
                                  x=list(correlation_matrix.columns),
                                  y=list(correlation_matrix.index),
                                  colorscale='Viridis')

fig.update_layout(title='Matriz de Correlação',
                  xaxis=dict(title='Variáveis'),
                  yaxis=dict(title='Variáveis'),
                  coloraxis_colorbar=dict(title='Correlação'))

fig.show()

<ipython-input-56-5d9efc08908b>:3: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



# Observando dados referentes ao segundo turno

Nas demais células, estão replicadas as análises acima que são pertinentes para observar os dados vinculados ao segundo turno das eleições.

In [57]:
df_segundo_turno = pd.read_csv("SEGUNDO_TURNO_CG_dados_tratados.csv").drop(columns=["Unnamed: 0", "NR_ZONA.1", "NR_SECAO.1"])
df_segundo_turno.head()

,NM_MUNICIPIO,NR_ZONA,NR_SECAO,NR_TURNO,NR_LOCAL_VOTACAO,NR_PARTIDO,SG_PARTIDO,DS_CARGO_PERGUNTA,DS_TIPO_VOTAVEL,NM_VOTAVEL,NR_LATITUDE,NR_LONGITUDE,NM_BAIRRO,NM_LOCAL_VOTACAO
0,CAMPINA GRANDE,72,66,2.0,1198.0,-1.0,#NULO#,Presidente,Branco,Branco,-7.226579,-35.918852,CONJUNTO SEVERINO CABRAL,ESCOLA ESTADUAL DE 1. E 2. GRAUS SEVERINO CABRAL
1,CAMPINA GRANDE,72,66,2.0,1198.0,22.0,PL,Presidente,Nominal,JAIR BOLSONARO,-7.226579,-35.918852,CONJUNTO SEVERINO CABRAL,ESCOLA ESTADUAL DE 1. E 2. GRAUS SEVERINO CABRAL
2,CAMPINA GRANDE,72,66,2.0,1198.0,13.0,PT,Presidente,Nominal,LULA,-7.226579,-35.918852,CONJUNTO SEVERINO CABRAL,ESCOLA ESTADUAL DE 1. E 2. GRAUS SEVERINO CABRAL
3,CAMPINA GRANDE,72,66,2.0,1198.0,-1.0,#NULO#,Presidente,Nulo,Nulo,-7.226579,-35.918852,CONJUNTO SEVERINO CABRAL,ESCOLA ESTADUAL DE 1. E 2. GRAUS SEVERINO CABRAL
4,CAMPINA GRANDE,72,66,2.0,1198.0,40.0,PSB,Governador,Nominal,JOÃO,-7.226579,-35.918852,CONJUNTO SEVERINO CABRAL,ESCOLA ESTADUAL DE 1. E 2. GRAUS SEVERINO CABRAL


## Densidade de votos por local de votação

O código abaixo gera um mapa de densidade que mostra a distribuição geográfica da densidade de votos por local de votação em Campina Grande, usando as coordenadas de latitude e longitude. Cada ponto no mapa representa um local de votação e a densidade de votos é indicada pela intensidade da cor.

In [58]:
coord_counts = df_segundo_turno.groupby(['NR_LATITUDE', 'NR_LONGITUDE']).size().reset_index(name='Contagem')

fig = px.density_mapbox(coord_counts, lat='NR_LATITUDE', lon='NR_LONGITUDE', z='Contagem', radius=30,
                        center=dict(lat=-7.2307, lon=-35.8811), zoom=12,
                        hover_name='Contagem', hover_data=['NR_LATITUDE', 'NR_LONGITUDE'],
                        title='Distribuição Geográfica da densidade de votos por local de votação em Campina Grande',
                        mapbox_style='open-street-map')

fig.show()

## Distribuição das Zonas Eleitorais

In [59]:
agrupamento_zona = df_segundo_turno.groupby(['NR_ZONA', 'NR_LATITUDE', 'NR_LONGITUDE']).size().reset_index(name='Contagem')
agrupamento_zona['Zona'] = agrupamento_zona['NR_ZONA'].astype(str)

fig = px.scatter_mapbox(agrupamento_zona, lat='NR_LATITUDE', lon='NR_LONGITUDE', color='Zona', size='Contagem',
                        center=dict(lat=-7.2307, lon=-35.8811), zoom=12,
                        hover_name='NR_ZONA', hover_data=['Contagem'],
                        title='Distribuição das Zonas Eleitorais na cidade de Campina Grande',
                        mapbox_style='open-street-map',
                        color_discrete_sequence=color_pallete,
                        color_discrete_map={i: f'Zona {i}' for i in agrupamento_zona['NR_ZONA'].unique()},
                        size_max=30)

fig.update_layout(
    legend=dict(
        title='Zonas Eleitorais',
        orientation='h',
        itemsizing='constant'
    )
)

fig.show()

## Contagem de votos nos Top 10 partidos por bairro

In [60]:
cores_partidos = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2',
                  '#7f7f7f', '#bcbd22', '#17becf', '#b5bd61', '#c49c94', '#f7b6d2', '#c7c7c7',
                  '#dbdb8d', '#9edae5', '#98df8a', '#ff9896', '#aec7e8', '#ffbb78', '#c5b0d5',
                  '#ff9896', '#ff7f0e', '#aec7e8', '#ffbb78', '#98df8a', '#c49c94', '#dbdb8d',
                  '#ffbb78', '#ff9896', '#aec7e8', '#ff7f0e', '#98df8a']

In [61]:
agrupamento_partido_bairro = df_segundo_turno.groupby(['SG_PARTIDO', 'NM_BAIRRO']).size().reset_index(name='Contagem')

agrupamento_partido_bairro = agrupamento_partido_bairro.sort_values(by='Contagem', ascending=False)

top_10_partidos_bairro = agrupamento_partido_bairro.groupby('NM_BAIRRO').head(10)

fig = px.bar(top_10_partidos_bairro, x='NM_BAIRRO', y='Contagem', color='SG_PARTIDO',
             title='Contagem de Votos para os Top 10 Partidos por Bairro',
             labels={'Contagem': 'Número de Votos', "NM_BAIRRO": 'Bairro', "SG_PARTIDO": "Partido"},
             color_discrete_sequence=px.colors.qualitative.Plotly)

fig.update_layout(legend_title='Partido', barmode='stack')
fig.show()


## Contagem de votos brancos e nulos por bairro

Essa visualização é útil para entender a distribuição de votos brancos e nulos por zona eleitoral em cada bairro. A empilhamento das barras permite comparar a contagem total de votos brancos e nulos por bairro e observar como cada tipo de voto contribui para esse total. A legenda de cores indica se a barra representa um voto Branco ou Nulo.

In [62]:
df_brancos_nulos_zona = df_segundo_turno[df_segundo_turno['DS_TIPO_VOTAVEL'].isin(['Branco', 'Nulo'])]

agrupamento_zona_tipo_voto = df_brancos_nulos_zona.groupby(['NM_BAIRRO', 'DS_TIPO_VOTAVEL']).size().reset_index(name='Contagem')

fig = px.bar(agrupamento_zona_tipo_voto, x='NM_BAIRRO', y='Contagem', color='DS_TIPO_VOTAVEL',
             title='Contagem de Votos Brancos e Nulos por Zona Eleitoral',
             labels={'Contagem': 'Número de Votos', "NM_BAIRRO": 'Zona Eleitoral', "DS_TIPO_VOTAVEL": 'Tipo de Voto'},
             color_discrete_sequence=px.colors.qualitative.Plotly)

fig.update_layout(legend_title='Tipo de Voto')

fig.show()

## Calculando as correlações entre as colunas

In [63]:
import plotly.figure_factory as ff

correlation_matrix = df_segundo_turno.drop(columns=["NR_TURNO"]).corr()

fig = ff.create_annotated_heatmap(z=correlation_matrix.values,
                                  x=list(correlation_matrix.columns),
                                  y=list(correlation_matrix.index),
                                  colorscale='Viridis')

fig.update_layout(title='Matriz de Correlação',
                  xaxis=dict(title='Variáveis'),
                  yaxis=dict(title='Variáveis'),
                  coloraxis_colorbar=dict(title='Correlação'))

fig.show()

<ipython-input-63-08e7c1bd3c4b>:3: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.

